In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2

%matplotlib inline

In [ ]:
hub.KerasLayer(
    "https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/050-224-classification/2",
    trainable=True,
    arguments=dict(batch_norm_momentum=0.997),
)

In [ ]:
import tf_keras

IMAGE_SHAPE = (224, 224)
classifier = tf_keras.Sequential(
    [
        hub.KerasLayer(
            "https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/050-224-classification/2",
            input_shape=IMAGE_SHAPE + (3,),
        )
    ]
)

In [ ]:
import pathlib

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(
    "flower_photos", origin=dataset_url, untar=True, cache_dir="."
)
data_dir = pathlib.Path(f"{data_dir}/flower_photos")

In [ ]:
list(data_dir.glob("*/*.jpg"))[:5]  # Display some of the images in the dataset

In [ ]:
roses = list(data_dir.glob("roses/*"))
tulips = list(data_dir.glob("tulips/*"))
daisy = list(data_dir.glob("daisy/*"))
dandelion = list(data_dir.glob("dandelion/*"))
sunflowers = list(data_dir.glob("sunflowers/*"))

In [ ]:
flowers_image_dict = {
    "roses": roses,
    "daisy": daisy,
    "dandelion": dandelion,
    "sunflowers": sunflowers,
    "tulips": tulips,
}

In [ ]:
flowers_labels_dict = {
    "roses": 0,
    "daisy": 1,
    "dandelion": 2,
    "sunflowers": 3,
    "tulips": 4,
}

In [ ]:
X, y = [], []
for flower_name, images in flowers_image_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, IMAGE_SHAPE, interpolation=cv2.INTER_LANCZOS4)
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [ ]:
import numpy as np

X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
predicted = classifier.predict(np.array([X[0], X[1], X[2]]))
predicted = np.argmax(predicted, axis=-1)
predicted

In [ ]:
with open("ImageNetLabels.txt", "r") as f:
    labels = f.read().splitlines()

In [ ]:
labels[predicted[0]], labels[predicted[1]], labels[predicted[2]]

In [ ]:
feature_extractor_url = "https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/050-224-feature-vector/2"

In [ ]:
pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_url, input_shape=IMAGE_SHAPE + (3,), trainable=False
)

In [ ]:
model = tf_keras.Sequential(
    [
        pretrained_model_without_top_layer,
        tf_keras.layers.Dense(5, activation="softmax"),
    ]
)
model.summary()

In [ ]:
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
model.fit(X_train, y_train, epochs=20)

In [ ]:
model.evaluate(X_test, y_test)